# Multi-armed bandits (MAB) and the cold-start scenario. Comparison with hybrid recommenders.

In [ ]:
import os
username = 'recspert'
repo = 'ITP-SeqRecSys-2024'

# remove local directory if it already exists
if os.path.isdir(repo):
    !rm -rf {repo}

!git clone https://github.com/{username}/{repo}.git

In [ ]:
!pip install --no-cache-dir --upgrade git+https://github.com/evfro/polara.git@develop#egg=polara

In [ ]:
!pip install lightfm

In [ ]:
import numpy as np
import pandas as pd

from polara import get_movielens_data
from polara.preprocessing.dataframes import reindex

%cd {repo}
from source.dataprep.dataprep import transform_indices, generate_interactions_matrix
from source.evaluation.evaluation import topn_recommendations
%cd -

from lightfm import LightFM

from scipy.sparse import csr_matrix, identity, hstack
import matplotlib.pyplot as plt
import os, time
from tqdm.notebook import tqdm


In [ ]:
!curl https://files.grouplens.org/datasets/movielens/ml-1m.zip --output ml-1m.zip

In [ ]:
!unzip ml-1m.zip

In [ ]:
data_path = './ml-1m/'
data = pd.read_csv(os.path.join(data_path, 'ratings.dat'), sep='::', names=['userid', 'movieid', 'rating', 'timestamp'], header=None, engine='python')
data['rating'] = (data['rating'] >= 4).astype(int)
user_data = pd.read_csv(os.path.join(data_path, 'users.dat'), sep='::', names=['userid', 'gender', 'age', 'occupation', 'zip-code'], header=None, engine='python')
item_data = pd.read_csv(os.path.join(data_path, 'movies.dat'), sep='::', names=['movieid', 'title', 'genres'], header=None, engine='python', encoding='latin-1')

In [ ]:
top_items = data['movieid'].value_counts().index[:100]
data = data[data['movieid'].isin(top_items)]

In [ ]:
data_description = {
    'items':'movieid',
    'users':'userid',
    'feedback':'rating',
    'timestamp':'timestamp'
}

In [ ]:
test_users = np.random.choice(data[data_description['users']].unique(), 500, replace=False)

test_data_ = data.query('userid in @test_users')
train_data_ = data.query('userid not in @test_users')

training, data_index = transform_indices(train_data_.copy(), 'userid', 'movieid')

genres_split = item_data['genres'].str.get_dummies(sep='|')
item_features_ = pd.concat([item_data['movieid'], genres_split], axis=1)
item_features_train = reindex(item_features_, data_index['items']).set_index(data_description['items'])

gender = pd.get_dummies(user_data['gender']).astype(int)
occupation = pd.get_dummies(user_data['occupation'], prefix='occupation').astype(int)
user_features_ = pd.concat([user_data['userid'], gender, occupation], axis=1)
user_features_train_ = user_features_[user_features_[data_description['users']].isin(train_data_[data_description['users']].unique())]
user_features_test_ = user_features_[user_features_[data_description['users']].isin(test_users)]

user_idx = pd.Index(
        test_data_[data_description['users']].unique(),
        dtype=data_index['users'].dtype,
        name=data_description['users'],
        )
user_features_train = reindex(user_features_train_, data_index['users']).set_index(data_description['users'])
user_features_test = reindex(user_features_test_, user_idx).set_index(data_description['users'])


testset = reindex(test_data_, [user_idx, data_index['items']])

In [ ]:
user_features_.head()

In [ ]:
item_features_.head()

In [ ]:
testset

[mab2rec](https://github.com/fidelity/mab2rec), [mabwiser](https://github.com/fidelity/mabwiser)

[openbandit](https://github.com/st-tech/zr-obp)

## Epsilon greedy

With probability $\varepsilon$ do eploration (pull a random arm), with probability $1 - \varepsilon$ do eploitation (choose the best arm)

In [ ]:
class epsGreedy:
    def __init__(self, config) -> None:
        self.n_arms = config['n_arms']
        self.epsilon = config['epsilon']
        self.seed = config['seed']
        self.rng = np.random.default_rng(seed=config['seed'])
        self.Q = np.zeros(config['n_arms'])
        self.click = np.zeros(config['n_arms'])
        
    def train(self, data, data_description, features):
        ...
    
    def choose_arm(self, context):
        ...
        return arm
    
    def update(self, arm, context, reward, update):
        ...
        
    def compute_probs(self, context):
        scores = np.tile(self.Q, (context.shape[0], 1))
        return scores

## LinUCB



$$
\theta_a = A_a^{-1}b_a
$$

$$
p_{t,a} = \theta_a^\top x_{t,a} + \alpha \sqrt{x_{t,a}^\top A_a^{-1} x_{t,a}}
$$

Choose arm with highest $p_{t, a}$, observe reward $r_t$.

$$
A_a \leftarrow A_a + x_{t,a}x_{t,a}^\top
$$

$$
b_a \leftarrow b_a + r_tx_{t,a}
$$

In [ ]:
class LinUCB:
    def __init__(self, config):
        self.n_arms = config['n_arms']
        self.d = config['context_dim']
        self.alpha = config['alpha']
        self.seed = config['seed']
        self.rng = np.random.default_rng(seed=config['seed'])
        
        self.A = np.array([np.eye(config['context_dim']) for _ in range(config['n_arms'])])
        self.b = np.array([np.zeros(config['context_dim']) for _ in range(config['n_arms'])])
        self.A_inv = np.array([np.eye(config['context_dim']) for _ in range(config['n_arms'])])
        self.theta = np.zeros((config['n_arms'], config['context_dim']))

    def train(self, data, data_description, features):
        occurences = data.groupby(data_description['items']).size()
        positives = data.groupby(data_description['items'])[data_description['feedback']].sum()
        
        for item in occurences.index:
            self.A[item] += np.outer(features.loc[item].values, features.loc[item].values) * occurences.loc[item]
            self.b[item] += features.loc[item].values * positives.loc[item]
            
        self.A_inv = np.linalg.inv(self.A)
        self.theta = np.einsum('ijk,ik->ij', self.A_inv, self.b, optimize=True)
        

    def choose_arm(self, features):
        n_items = self.A.shape[0]
        probs = np.zeros(n_items)

        probs = np.einsum('ij,ij->i', self.theta, features.values, optimize=True) + self.alpha * np.sqrt(
            np.einsum('ijk,ij,ik->i', self.A_inv, features.values, features.values, optimize=True)
        )

        chosen_arm = np.argmax(probs)
        
        return chosen_arm
    
    def update(self, arm, features, reward, update=True):
        ...
            
    def compute_probs(self, features):
        n_items = self.A.shape[0]
        probs = np.zeros(n_items)

        probs = np.einsum('ij,ij->i', self.theta, features.values, optimize=True) + self.alpha * np.sqrt(
            np.einsum('ijk,ij,ik->i', self.A_inv, features.values, features.values, optimize=True)
        )
        
        return probs

# Stream evaluation

In [ ]:
def stream_evaluate(Bandit, bandit_cfg, data, data_description, batchsize=100):
    bandit = Bandit(bandit_cfg)
    bandit.train(data['train'], data_description, data['features'])

    # stream of events
    current_reward = 0
    hr = 0
    unique = set()
    
    step = 1
    rewards_history = []
    for i in range(len(data['interactions'])):
        item = data['interactions'].iloc[i][data_description['items']]
        reward = data['interactions'].iloc[i][data_description['feedback']]

        arm = bandit.choose_arm(data['features'])
        
        if arm == item:
            current_reward += reward
            hr += 1
            unique.add(item)

        rewards_history.append(current_reward / step)
        step += 1
        
        bandit.update(arm, data['features'], reward, update=(i + 1) % batchsize == 0)
    print('hr: ', hr / step, 'cov: ', len(unique) / bandit.n_arms)
    return bandit, rewards_history
        

In [ ]:
bandit_data = {
    'train':training,
    'interactions':testset.sort_values(by=data_description['timestamp']),
    'features':item_features_train,
    'user_features':user_features_train,
    'user_features_test':user_features_test
}

In [ ]:
ucb_results = {}
for alpha in [-0.5, 0.1, 1.0, 5.0]:
    ucb_cfg = {
        'n_arms':bandit_data['interactions'].nunique()[data_description['items']],
        'context_dim':bandit_data['features'].shape[1],
        'alpha':alpha,
        'seed':2024
    }
    ucb, ucb_rewards = stream_evaluate(LinUCB, ucb_cfg, bandit_data, data_description, batchsize=100)
    plt.plot(ucb_rewards, label=alpha)
    ucb_results[alpha] = ucb_rewards
    
plt.legend()
plt.show()

In [ ]:
eps_results = {}
for epsilon in [0.1, 0.5, 0.9]:
    eps_cfg = {
        'n_arms':bandit_data['interactions'].nunique()[data_description['items']],
        'epsilon':epsilon,
        'seed':2024
    }
    eps, eps_rewards = stream_evaluate(epsGreedy, eps_cfg, bandit_data, data_description, batchsize=100)
    plt.plot(eps_rewards, label=epsilon)
    eps_results[epsilon] = eps_rewards
    
plt.legend()
plt.show()

# LightFM

[Code](https://github.com/lyst/lightfm/tree/master), [paper](https://arxiv.org/pdf/1507.08439)

$$
q_u = \sum_{j\in f_u}e^U_j, \ \ \ b_u = \sum_{j\in f_u}b^U_j
$$

$$
p_i = \sum_{j\in f_i}e^I_j, \ \ \ b_i = \sum_{j\in f_i}b^I_j
$$

$$
r_{ui} = \sigma(q_u^\top p_i + b_u + b_i)
$$

In [ ]:
def model_evaluate_coldstart(recommended_items, holdout, holdout_description, topn=10):
    itemid = holdout_description['items']
    holdout_items = holdout[itemid].values
    assert recommended_items.shape[0] == len(holdout[itemid].nunique())
    hits_mask = recommended_items[:, :topn] == holdout_items.reshape(-1, 1)
    # HR calculation
    hr = np.mean(hits_mask.any(axis=1))
    # MRR calculation
    n_test_items = recommended_items.shape[0]
    hit_rank = np.where(hits_mask)[1] + 1.0
    mrr = np.sum(1 / hit_rank) / n_test_items
    # coverage calculation
    n_users = holdout_description['n_users']
    cov = np.unique(recommended_items).size / n_users
    return hr, mrr, cov


def check_early_stop_config_coldstart(early_stop_config):
    """
    Validates the early stop configuration and returns a config dictionary.

    Parameters
    ----------
    early_stop_config : dict, optional
        Dictionary containing the early stop configuration.

    Returns
    -------
    es_dict : dict
        Dictionary containing the early stop configuration, or a dictionary
        with 'stop_early' set to False if no valid configuration is provided.
    """
    if early_stop_config is None:
        early_stop_config = {}
    try:
        es_dict = {
            'early_stopper': early_stop_config['evaluation_callback'],
            'callback_interval': early_stop_config['callback_interval'],
            'item_features': early_stop_config['item_features'],
            'train': early_stop_config['train'],
            'holdout': early_stop_config['holdout'],
            'target_metric': early_stop_config['target_metric'],
            'stop_early': True
        }
    except KeyError: # config is invalid, doesn't contain required keys
        es_dict = {'stop_early': False} # disable early stopping
    return es_dict


def lfm_best_model_search(data, data_description, config):
    data_descr_lfm = dict(
        users = data_description['users'],
        items = data_description['items'],
        feedback = data_description['feedback'],
        n_users = data['interactions'].shape[0],
        n_items = data['interactions'].shape[1],
        user_features = hstack(
            [
                identity(data['interactions'].shape[0], dtype=data['item_features'].dtype, format='csr'),
                data['user_features']
                ]).tocsr(),
        item_features = hstack(
            [
                identity(data['interactions'].shape[1], dtype=data['user_features'].dtype, format='csr'),
                data['item_features']
                ]).tocsr(),
    )

    lfm_params = build_lfm_model(
        config,
        data['interactions'],
        data_descr_lfm,
        early_stop_config = None
    )
    return lfm_params

def build_lfm_model(config, data, data_description, early_stop_config=None, iterator=None):

    model = LightFM(
        no_components = config['no_components'],
        loss = config['loss'],
        learning_schedule = config['learning_schedule'],
        # learning_rate=
        user_alpha = config['user_alpha'],
        item_alpha = config['item_alpha'],
        max_sampled = config['max_sampled'],
        # random_state = 
    )
    # early stoppping configuration
    es_config = check_early_stop_config_coldstart(early_stop_config)

    # training
    if iterator is None:
        iterator = lambda x: x
    for epoch in iterator(range(config['max_epochs'])):
        try:
            train_lfm_epoch(epoch, model, data, data_description, es_config)
        except StopIteration:
            break
    return model

def train_lfm_epoch(
    epoch, model, train, data_description, es_config,
):

    model.fit_partial(
        train,
        user_features = data_description['user_features'],
        item_features = data_description['item_features'],
        epochs = 1,
        num_threads=80
    )
    if not es_config['stop_early']:
        return
    metrics_check_interval = es_config['callback_interval']
    if (epoch+1) % metrics_check_interval == 0:
        # evaluate model and raise StopIteration if early stopping condition is met
        
        early_stopper_call = es_config['early_stopper']
        early_stopper_call(epoch, model, es_config['user_features'], es_config['item_features'], es_config['holdout'], data_description, es_config['target_metric'])
        
        
def lightfm_scoring(model, user_features, item_features, data_description):
    """
    A standard scoring function adopted for use with LightFM in the item cold-start settings.
    It returns a 2D item-user array (i.e., a transposed matrix of interactions) corresponding
    to the predicted scores of user relevance to cold items.
    """    
    user_biases, user_embeddings = model.get_user_representations(features=user_features)
    item_biases, item_embeddings = model.get_item_representations(features=item_features)
    scores = user_embeddings @ item_embeddings.T  + item_biases

    return scores


def lfm_evaluator(model, user_features, item_features, holdout, data_description, target_metric):
    """
    Helper function to run within an evaluation callback.
    
    Intended usage:
    - in the early stopping setting for tuning based on a `target_metric`.
    """
    holdout_users = holdout[data_description['users']].values

    lfm_scores = lightfm_scoring(model, user_features[holdout_users, :], item_features, data_description)
    lfm_recs = topn_recommendations(lfm_scores)
    metrics = model_evaluate_coldstart(lfm_recs, holdout, data_description)
    return metrics[target_metric]

In [ ]:
data_description['n_users'] = training.nunique()[data_description['users']]
data_description['n_items'] = training.nunique()[data_description['items']]

In [ ]:
interactions_matrix = generate_interactions_matrix(training, data_description)

lfm = lfm_best_model_search(
    {
        'interactions':interactions_matrix,
        'item_features':csr_matrix(item_features_train.values),
        'user_features':csr_matrix(user_features_train.values),
        'n_users':data_description['n_users'],
        'n_items':data_description['n_items'],
    },
    data_description,
    dict(
    no_components = 64,
    loss = 'warp',
    max_sampled = 3,
    max_epochs = 10,
    learning_schedule = 'adagrad',
    user_alpha = 1e-5,
    item_alpha = 1e-5,
    )
)

In [ ]:
def stream_evaluate_lfm(lfm, data, data_description, batchsize=100):
    # stream of events
    current_reward = 0
    hr = 0
    unique = set()
    
    step = 1
    rewards_history = []
    interactions_matrix = generate_interactions_matrix(data['train'], data_description)
    item_features_test = hstack(
        [
        identity(interactions_matrix.shape[1], format='csr'),
        csr_matrix(data['features'].values),
        ]).tocsr()

    for i in range(len(data['interactions'])):
        item = data['interactions'].iloc[i][data_description['items']]
        user = data['interactions'].iloc[i][data_description['users']]
        reward = data['interactions'].iloc[i][data_description['feedback']]
        
        user_features = hstack(
        [
            csr_matrix(([], ([], [])), shape=(1, interactions_matrix.shape[0])),
            csr_matrix(data['user_features_test'].values[user, :]),
        ]).tocsr()

        arm = np.argmax(lightfm_scoring(lfm, user_features, item_features_test, data_description))
        
        if arm == item:
            current_reward += reward
            hr += 1
            unique.add(item)

        rewards_history.append(current_reward / step)
        step += 1
        
    print('hr: ', hr / step, 'cov: ', len(unique) / data_description['n_items'])
    return rewards_history

In [ ]:
lfm_rewards = stream_evaluate_lfm(lfm, bandit_data, data_description)

In [ ]:
plt.plot(lfm_rewards, label='LightFM', c='black')

for alpha in ucb_results:
    plt.plot(ucb_results[alpha], label=f'UCB_{alpha}')

for epsilon in eps_results:
    plt.plot(eps_results[epsilon], label=f'EPS_{epsilon}', ls='dashed')
    
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.ylim(0.0, 0.02)
plt.show()

# Off-policy evaluation

$$
\hat{V}^{\text{ips}}_e = \frac{1}{n} \sum_{i = 1}^n \frac{\pi_e(a_i|x_i)}{\pi_0(a_i|x_i)}r_i
$$

In [ ]:
def train_offpolicy_ips(Bandit, bandit_cfg, data, data_description, batchsize=10000):
    bandit = Bandit(bandit_cfg)
    bandit.train(data['train'], data_description, data['features'])

    # stream of events
    v_ips = 0
    step = 1
    ips_scores = []
    for i in range(len(data['interactions'])):
        if (i + 1) % batchsize == 0:
            ips_scores.append(v_ips / step)
            v_ips = 0
            step = 1
        item = data['interactions'].iloc[i][data_description['items']]
        reward = data['interactions'].iloc[i][data_description['feedback']]
        
        v_ips += reward * bandit.compute_probs(data['features'])[item] / (1 / bandit.n_arms)

        step += 1
    return bandit, ips_scores
        

In [ ]:
ucb_cfg = {
    'n_arms':bandit_data['interactions'].nunique()[data_description['items']],
    'context_dim':bandit_data['features'].shape[1],
    'alpha':1.0,
    'seed':2024
}

ucb, ips_scores_ucb = train_offpolicy_ips(LinUCB, ucb_cfg, bandit_data, data_description, batchsize=100)

In [ ]:
print(f'{np.mean(ips_scores_ucb)} +- {np.std(ips_scores_ucb)}')

In [ ]:
eps_cfg = {
    'n_arms':bandit_data['interactions'].nunique()[data_description['items']],
    'epsilon':0.5,
    'seed':2024
}
eps, ips_scores_eps = train_offpolicy_ips(epsGreedy, eps_cfg, bandit_data, data_description, batchsize=100)


In [ ]:
print(f'{np.mean(ips_scores_eps)} +- {np.std(ips_scores_eps)}')

In [ ]:
def offpolicy_ips(lfm, data, data_description, batchsize=100):
    # stream of events
    v_ips = 0
    step = 1
    ips_scores = []

    interactions_matrix = generate_interactions_matrix(data['train'], data_description)
    item_features_test = hstack(
        [
        identity(interactions_matrix.shape[1], format='csr'),
        csr_matrix(data['features'].values),
        ]).tocsr()

    for i in range(len(data['interactions'])):
        if (i + 1) % batchsize == 0:
            ips_scores.append(v_ips / step)
            v_ips = 0
            step = 1
        item = data['interactions'].iloc[i][data_description['items']]
        user = data['interactions'].iloc[i][data_description['users']]
        reward = data['interactions'].iloc[i][data_description['feedback']]
        
        user_features = hstack(
            [
                csr_matrix(([], ([], [])), shape=(1, interactions_matrix.shape[0])),
                csr_matrix(data['user_features_test'].values[user, :]),
            ]).tocsr()
        lfm_scores = lightfm_scoring(lfm, user_features, item_features_test, data_description).squeeze()
        lfm_scores[lfm_scores < 0] = 0.0
        lfm_scores = lfm_scores / lfm_scores.sum()
        
        v_ips += reward * lfm_scores[item] / (1 / 100)

        step += 1
        
    return ips_scores

In [ ]:
ips = offpolicy_ips(lfm, bandit_data, data_description, batchsize=100)
print(f'{np.mean(ips)} +- {np.std(ips)}')